In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from setupDataset import get_dataset, load
from feature_vector import get_feature_names
import numpy as np
from Utils.univariate_featureSelection import get_selectedFeatures, get_metrics, metrics_feature_selection

def metrics_feature_selection(ranks, X_train, y_train, X_test, y_test, n):

calls get_metrics

In [ ]:
def featureSelectionResults(trainSet, validationSet, rankfile='rank_selections.cvs', RFE_rankfile='RFE_ranks.csv', n=25, k=5):
    # 1. get the feature rankings
    # 1.1 univariate ranks from file
    ranks = getTopFeatures(trainSet, rankfile)
    
    # 1.2 compute RFE ranks
    RFE_ranks = getRFE_ranking(dataset, RFE_rankfile)
    ranks = np.concatenate((ranks, RFE_ranks), axis=1)
    
    # 2. separate into test validation sets    
    good, bad = separateByCategory(dataset)
    # good, bad, validate = get_testSet_validationSet(good, bad) brauch kein validation set momentan, oder?
    sets = getTestSets(good, bad, k) #  returns a list of k datasets (each  { "data": data, "target": target})
    r_len = len(ranks.T) + 1 # +1 for result without feature selection
    metrics = [np.zeros(8) for i in range(r_len)]
    
    for index, set in enumerate(sets):
        #actually each set is once the testing set, and then the others are used for training
        # 1. the index set is the testing set, 
        X_train, y_train, X_test, y_test = get_train_test_set(sets, index)

        
        metrics_temp = metrics_feature_selection(ranks, X_train, y_train, X_test, y_test, n)
        metrics = [metrics[i]+metrics_temp[i] for i in range(r_len)] #sum up the metrics (compute avg later)

    
    # now divide all values of the metrics (except TN, TP, FN, FP) by the number of sets
    set_len = len(sets)
    avg_mask = np.array([set_len, set_len, set_len, set_len, 1, 1, 1, 1])
    metrics = [m/avg_mask for m in metrics] #divide by the number of sets to get the average metrics
    
    return SelectionMetrics(metrics, ranks, n)